<a id="introduction"></a>
## Introduction to Dimensionality Reduction
#### By Paul Hendricks
-------

In this notebook, we will show how to do GPU accelerated Dimensionality Reduction in RAPIDS.

**Table of Contents**

* [Introduction to Dimensionality Reduction](#introduction)
* [Principle Components Analysis](#pca)
* [Truncated SVC](#tvsd)
* [UMAP](#umap)
* [Setup](#setup)
* [Conclusion](#conclusion)

Before going any further, let's make sure we have access to `matplotlib`, a popular Python library for visualizing data.

In [ ]:
import os

try:
    import matplotlib
except ModuleNotFoundError:
    os.system('conda install -y matplotlib')

<a id="setup"></a>
## Setup

This notebook was tested using the following Docker containers:

* `rapidsai/rapidsai-nightly:0.8-cuda10.0-devel-ubuntu18.04-gcc7-py3.7` from [DockerHub - rapidsai/rapidsai-nightly](https://hub.docker.com/r/rapidsai/rapidsai-nightly)

This notebook was run on the NVIDIA Tesla V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

If you think you have found a bug or an error, please file an issue here: https://github.com/rapidsai/notebooks/issues

Before we begin, let's check out our hardware setup by running the `nvidia-smi` command.

In [ ]:
!nvidia-smi

Next, let's see what CUDA version we have:

In [ ]:
!nvcc --version

Next, let's load some helper functions from `matplotlib` and configure the Jupyter Notebook for visualization.

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt


%matplotlib inline

<a id="Data"></a>
## Data

We'll start by loading the Digits dataset. This dataset is composed of 1,797 images of hand written digits, each of shape 8 x 8 (64 total columns when flattended) .

In [ ]:
import numpy as np; print('NumPy Version:', np.__version__)
import sklearn; print('Scikit-Learn Version:', sklearn.__version__)
from sklearn.datasets import load_digits


digits = load_digits()
X, y = digits['data'], digits['target']
X, y = X.astype(np.float32), y.astype(np.float32)
print('X: ', X.shape, X.dtype, 'y: ', y.shape, y.dtype)

Let's visualize several examples from our dataset:

In [ ]:
# create figure
figure = plt.figure()
f, axes = plt.subplots(4, 4, figsize=(10, 10))

i = 0
for row in axes:
    for axis in row:
        axis.imshow(X[i].reshape(8, 8), cmap='gray')
        axis.set_title('Class: ' + str(int(y[i])))
        i += 1
    
plt.tight_layout()
plt.show()

<a id="pca"></a>
## Principle Components Analysis

PCA (Principal Component Analysis) is a fundamental dimensionality reduction technique used to combine features in X in linear combinations such that each new component captures the most information or variance of the data. `n_components`is usually small, say at 3, where it can be used for data visualization, data compression and exploratory analysis.

cuML’s `PCA` expects a cuDF DataFrame, and provides 2 algorithms Full and Jacobi. Full (default) uses a full eigendecomposition then selects the top K eigenvectors. The Jacobi algorithm is much faster as it iteratively tries to correct the top K eigenvectors, but might be less accurate.

Before we fit a PCA model to the data, we'll standardize the data using Scikit-Learn's `StandardScaler` class.

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

We can fit this object to the dataset using the `fit` and `transform` methods.

In [ ]:
scaler.fit(X)
X_scaled = scaler.transform(X)

Next, we'll import the `PCA` class from Scikit-Learn and instantiate a model.

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)

We can fit this model to our data and then use that estimated model to reduce our dataset to the 2 principle components that explain as much variation in the data as possible.

In [ ]:
pca.fit(X_scaled)
components = pca.transform(X_scaled)
components

Since we've reduced our dataset to 2 dimensions, we can easily visualize this dataset. The below code will plot each digit with its associated principle component values. Images within a class are visually similar; these images will appear within clusters. Additionally, classes that are visually similar (6s may look like 0s; 3s and 9s may look like 8s; etc.) are going to appear closer to each other. 

With this reduced dimensionality, it's easier to train an algorithm to classify an image into a class. Reducing the dimensionality of a dataset before building a classification or regression model is a common step in machine learning.

In [ ]:
# set colors
colors = ['blue', 'orange', 'green', 'red', 'purple', 
          'brown', 'pink', 'gray', 'olive', 'cyan']
colors = ['tab:' + color for color in colors]

# create figure
figure = plt.figure()
axis = figure.add_subplot(111)

for i in range(10):
    mask = y == i
    axis.scatter(components[mask, 0], components[mask, 1], 
                 c=colors[i], label=str(i))
axis.set_title('PCA (cpu)')

plt.legend()
plt.tight_layout()
plt.show()

The previous example used the Scikit-Learn implementation of PCA which is CPU-based. We can use a GPU accelerated version of this algorithm using the cuML library in RAPIDS.

First, let's convert our data from a NumPy array representation to a Pandas DataFrame.

In [ ]:
import pandas as pd; print('Pandas Version:', pd.__version__)


X_scaled_df = pd.DataFrame(X_scaled)
X_scaled_df.columns = ['feature_' + str(i) for i in range(X_scaled_df.shape[1])]

Next, we'll convert our Pandas DataFrame to a cuDF DataFrame.

In [ ]:
import cudf; print('cuDF Version:', cudf.__version__)


X_scaled_cudf = cudf.DataFrame.from_pandas(X_scaled_df)

Similar to Scikit-Learn, we can import a PCA model from cuML and instantiate it.

In [ ]:
import cuml; print('cuML Version:', cuml.__version__)
from cuml.decomposition import PCA as PCA_GPU


pca_gpu = PCA_GPU(n_components=2)

We can fit our PCA model to the data using the `fit` method and transform the dataset into principle components using the `transform` method.

In [ ]:
pca_gpu.fit(X_scaled_cudf)
components_gpu = pca_gpu.transform(X_scaled_cudf).to_pandas().values
components_gpu

Let's visualize the PCA components generated by the GPU model and compare them to those generated by the CPU model. They should be exactly the same!

In [ ]:
# create figure
figure = plt.figure()
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))

for i in range(10):
    mask = y == i
    ax1.scatter(components[mask, 0], components[mask, 1], 
                c=colors[i], label=str(i))
    ax2.scatter(components_gpu[mask, 0], components_gpu[mask, 1], 
                c=colors[i], label=str(i))
ax1.set_title('PCA (cpu)')
ax2.set_title('PCA (gpu)')
    
plt.legend()
plt.tight_layout()
plt.show()

<a id="tvsd"></a>
## Truncated SVD

TruncatedSVD is used to compute the top K singular values and vectors of a large matrix X. It is much faster when `n_components` is small, such as in the use of PCA when 3 components is used for 3D visualization.

cuML’s `TruncatedSVD` expects a cuDF DataFrame, and provides 2 algorithms Full and Jacobi. Full (default) uses a full eigendecomposition then selects the top K singular vectors. The Jacobi algorithm is much faster as it iteratively tries to correct the top K singular vectors, but might be less accurate.

Below, we import the `TruncatedSVD` from Scikit-Learn and instantiate it.

In [ ]:
from sklearn.decomposition import TruncatedSVD


tsvd = TruncatedSVD(n_components=2)

Previously, we fitted the model using the `fit` method and used the model to transform the dataset into components using the `transform`. Using the `fit_transform`, we can accomplish both of these steps in one line of code.

In [ ]:
components = tsvd.fit_transform(X)
components

Like before, we can visualize the components.

In [ ]:
# create figure
figure = plt.figure()
axis = figure.add_subplot(111)

for i in range(10):
    mask = y == i
    axis.scatter(components[mask, 0], components[mask, 1], 
                 c=colors[i], label=str(i))
axis.set_title('Truncated SVD (cpu)')

plt.legend()
plt.tight_layout()
plt.show()

Let's accomplish the same workflow using cuDF and cuML. We'll first prepare our data:

In [ ]:
X_df = pd.DataFrame(X)
X_df.columns = ['feature_' + str(i) for i in range(X_df.shape[1])]
X_cudf = cudf.DataFrame.from_pandas(X_df)

Next, we'll instantiate the GPU-accelerated version of `TruncatedSVD` from cuML.

In [ ]:
from cuml.decomposition import TruncatedSVD as TruncatedSVD_GPU


tvsd_gpu = TruncatedSVD_GPU(n_components=2)

We'll use the `fit_transform` method to fit the model and transform the data.

In [ ]:
components_gpu = tvsd_gpu.fit_transform(X_cudf).to_pandas().values
components_gpu

Lastly, we'll visualize the resulting components and show the CPU-based results and the GPU-based results side-by-side.

In [ ]:
# create figure
figure = plt.figure()
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))

for i in range(10):
    mask = y == i
    ax1.scatter(components[mask, 0], components[mask, 1], 
                c=colors[i], label=str(i))
    ax2.scatter(components_gpu[mask, 0], components_gpu[mask, 1], 
                c=colors[i], label=str(i))
ax1.set_title('Truncated SVD (cpu)')
ax2.set_title('Truncated SVD (gpu)')
    
plt.legend()
plt.tight_layout()
plt.show()

<a id="umap"></a>
## UMAP

Uniform Manifold Approximation and Projection Finds a low dimensional embedding of the data that approximates an underlying manifold.

We'll import the `UMAP` class from cuML and instantiate it.

In [ ]:
from cuml import UMAP as UMAP_GPU


umap_gpu = UMAP_GPU(n_neighbors=10, n_components=2)

We'll use the `fit_transform` method to fit the model and transform the data.

In [ ]:
components_gpu = umap_gpu.fit_transform(X_cudf).to_pandas().values
components_gpu

Lastly, we'll visualize the resulting components.

In [ ]:
# create figure
figure = plt.figure()
axis = figure.add_subplot(111)

for i in range(10):
    mask = y == i
    axis.scatter(components_gpu[mask, 0], components_gpu[mask, 1], 
                 c=colors[i], label=str(i))
axis.set_title('UMAP (gpu)')

plt.legend()
plt.tight_layout()
plt.show()

<a id="conclusion"></a>
## Conclusion

In this notebook, we showed how to do GPU accelerated Dimensionality Reduction in RAPIDS.

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)
